# Exploring the SQuAD dataset

In [1]:
# Test GPU
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2023-11-28 12:06:01.092076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 12:06:01.092112: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 12:06:01.092160: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 12:06:01.101866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 12:06:02.679865: I tensorflow/compiler/

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_v2")

explore_df = dataset["train"].to_pandas()

text = explore_df.head(1)["context"][0]
text

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

# Build a `keras` transformer model

## Imports

In [3]:
from keras_nlp.layers import TokenAndPositionEmbedding
from keras_nlp.metrics import Perplexity
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

Using TensorFlow backend


## Building vectorizer

In [46]:
context_list = list(explore_df["context"].unique()[:20])

In [47]:
# Maximum context length, in words
max_len = max([len(context.split()) for context in context_list])
max_len

312

In [48]:
context_list[0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [7]:
# vectorize_layer = TextVectorization(
#     standardize = "lower",
#     output_mode="int",
#     output_sequence_length=max_len,

# )
# vectorize_layer.adapt(context_list)

In [8]:
# vocab = vectorize_layer.get_vocabulary()

# vocab_size = len(vocab)
# vocab_size

In [9]:
# index_lookup = dict(zip(range(len(vocab)), vocab))
# index_lookup

In [10]:
# sentence_as_tokens = vectorize_layer(context_list[0])
# translated_back = [index_lookup[token] for token in sentence_as_tokens.numpy().tolist()]
# translated_back

In [11]:
# all_tokenized = [vectorize_layer(context) for context in context_list]

## Creating masked tokens & targets

In [12]:
# def X_y_creator(sequence_tensor):
#     # tile / copy the input sequence up to max_len -1
#     tiled_sequence = tf.tile(tf.expand_dims(sequence_tensor, 0), [max_len - 1, 1])
#     # use band_part to diagonally mask the sequence of tensors, so that there is a sequence of incremental strings
#     X_s = tf.linalg.band_part(tiled_sequence, -1, 0)
#     # y is simply the next word in the sequence
#     y_s = sequence_tensor[1:]
#     return X_s, y_s

In [13]:
# Xs = []
# ys = []
# for sequence in all_tokenized:
#     X, y = X_y_creator(sequence)
#     Xs.append(X)
#     ys.append(y)

In [14]:
# Xs[0]

In [15]:
# # Create a 2d tensor for X - this will have shape ((max_len - 1) * n_sequences, max_len)
# X = tf.concat(Xs, axis = 0)
# # Create a 1d tensor for y - this will have shape (max_len - 1)
# y = tf.concat(ys, axis=0)

In [16]:
# X[0]

In [17]:
# # Remove any sequences where the next predicted y is 0 - ie, where the next predicted word is null.
# X = X[y != 0]
# y = y[y != 0]

In [18]:
# y.shape

In [19]:
# X.shape

In [20]:
# embedding_dimensions = 768

# inputs = layers.Input(shape=(max_len,), dtype=tf.int32)
# x = TokenAndPositionEmbedding(vocab_size, max_len, embedding_dimensions, mask_zero=True)(inputs)
# x = layers.Dense(vocab_size, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(vocab_size, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(vocab_size, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(vocab_size, activation="relu")(x)
# x = tf.reduce_mean(x, axis=1)
# outputs = layers.Dense(vocab_size, activation="softmax")(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

# model.compile(
#     optimizer="adam",
#     loss="sparse_categorical_crossentropy",
#     metrics=[Perplexity(), "accuracy"]
# )

In [21]:
# model.summary()

In [22]:
# from tensorflow.keras.callbacks import EarlyStopping

# es = EarlyStopping(patience = 2, monitor="perplexity", restore_best_weights=True)

# history = model.fit(X, y, epochs = 2, workers= 5, callbacks=[es], batch_size=64)

## Predicting using first attempt model

In [23]:
# starting_word = "Born "

# tokens = vectorize_layer(starting_word)
# tokens_expanded = tf.expand_dims(tokens, 0)
# pred = model.predict(tokens_expanded)
# index_pred = pred.argmax()
# word = index_lookup[index_pred]
# word

In [24]:
# pred

# Second model attempt - with some BERT pre-training

In [49]:
from transformers import RobertaTokenizer, TFRobertaForMaskedLM
import tensorflow as tf
import numpy as np

# Load pre-trained model and tokenizer (without weights)
config = TFRobertaForMaskedLM.config_class.from_pretrained('roberta-base')
model = TFRobertaForMaskedLM(config)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.is_fast

False

In [50]:
# Get id of [MASK] token
mask_token_id = tokenizer.mask_token_id
mask_token_id

50264

In [51]:
{v:k for k,v in tokenizer.get_vocab().items()}[mask_token_id]

'<mask>'

In [52]:
context_list[0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [53]:
type(context_list)

list

In [55]:
def masked_sparse_categorical_crossentropy(y_true, y_pred):
    # Create a mask to ignore the -100 labels
    mask = tf.cast(tf.not_equal(y_true, -100), tf.float32)
    
    # Update y_true to a valid label value (e.g., 0) where it's -100
    y_true = tf.where(tf.not_equal(y_true, -100), y_true, tf.zeros_like(y_true))

    # Calculate the sparse categorical crossentropy
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

    # Apply the mask
    loss *= mask

    # Return the mean loss, but avoid division by zero
    return tf.reduce_sum(loss) / tf.maximum(tf.reduce_sum(mask), 1)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=masked_sparse_categorical_crossentropy)

# Tokenize input data and create a mask array
inputs = tokenizer(context_list, padding='max_length', truncation=True, return_tensors="tf", max_length=max_len)
input_ids = inputs.input_ids.numpy()

In [68]:
for i in range(1, 101):
    print(f"Random pass: {i}")
    # Regenerate random masks and retrain
    mask = np.random.rand(*input_ids.shape) < 0.15  # 15% probability of masking a token
    labels = -100 * np.ones(input_ids.shape, dtype=int)  # Initialize labels to -100
    labels[mask] = input_ids[mask]  # Assign labels for masked tokens
    # Apply mask
    input_ids[mask] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
    # Prepare dataset
    dataset = tf.data.Dataset.from_tensor_slices((
        {"input_ids": input_ids, "attention_mask": inputs.attention_mask, "labels": labels}
    )).batch(2)
    
    # Train the model
    history = model.fit(dataset, epochs=5)

Random pass: 1
Epoch 1/5
10/10 [==============================] - 1s 122ms/step - loss: 1.7190e-05
Epoch 2/5
10/10 [==============================] - 1s 120ms/step - loss: 1.7114e-05
Epoch 3/5
10/10 [==============================] - 1s 120ms/step - loss: 1.7003e-05
Epoch 4/5
10/10 [==============================] - 1s 120ms/step - loss: 1.6932e-05
Epoch 5/5
 9/10 [==========================>...] - ETA: 0s - loss: 1.6876e-05

KeyboardInterrupt: 

In [67]:
from transformers import pipeline

test_string = """Beyoncé <mask> Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy" """
pipe = pipeline("fill-mask", tokenizer=tokenizer, model=model, top_k=20)
pipe(test_string)

[{'score': 0.9999735355377197,
  'token': 50264,
  'token_str': '<mask>',
  'sequence': 'Beyoncé Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy" '},
 {'score': 7.831765742594143e-07,
  'token': 8,
  'token_str': ' and',
  'sequence': 'Beyoncé and Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer an

In [71]:
model.save_weights("checkpoint.h5")

In [72]:

model.load_weights("checkpoint.h5")